In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import lightgbm

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [ ]:
#데이터 확인

data_p.sample(10)

In [ ]:
data_p.info()
#결측값 없음 확인

In [ ]:
#불균형 분포 확인
print(data_p['voted'].value_counts())

#불균형 분포 아님을 확인

In [ ]:
#결손값 처리
data_p.isna().head(3)
print(data_p.isna().sum())

In [ ]:
####0값을 검사할 피처명 리스트 객체 설정


#무응답 해당 피처 : education, engnat, hand, urban
zero_features = ['education', 'engnat', 'hand', 'urban']
    
#전체 데이터 건수
total_count = data_p['education'].count()

#피처별로 반복 하면서 데이터 값이 0인 데이터 건수 추출하고, 퍼센트 계산
for feature in zero_features:
    zero_count = data_p[data_p[feature] == 0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature,  zero_count, 100* zero_count/total_count))
    
    
Q_E = []


#Q_E 시간
for i in range(97, 117):
    a = 'Q' + chr(i) + 'E'
    Q_E.append(a)
    
print(Q_E)
    
for feature in Q_E:
    zero_count = data_p[data_p[feature] == 0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature,  zero_count, 100* zero_count/total_count))

    
    
#tp
TP = []
for i in range(1,10):
        a = 'tp0' + str(i)
        TP.append(a)
TP.append('tp10')
print(TP)

for feature in TP:
    zero_count = data_p[data_p[feature] == 0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature,  zero_count, 100* zero_count/total_count)) 
    

In [ ]:
#0값을 평균값으로 대체
data_p[zero_features]=data_p[zero_features].replace(0, data_p[zero_features].mean())

In [ ]:
#Q_E 시간데이터 드랍 후 진행 - 성능평가 별 차이 없음
Q_E = []
for i in range(97, 117):
    a = 'Q' + chr(i) + 'E'
    Q_E.append(a)

for i in Q_E :
    vote_df.drop(i, axis=1, inplace=True)
    test_df.drop(i, axis=1, inplace=True)

In [ ]:
# 인덱스 제거
vote_df.drop("index", axis=1, inplace=True)
test_index = test_df["index"]
test_df.drop("index", axis=1, inplace=True)

In [ ]:
#데이터 변환
#원-핫 인코딩 대상 : gender, married, race, religion, 
#레이블 인코딩 대상 : age-group, 

age_group_=['10s', '20s', '30s', '40s', '50s', '60s', '+70s']

encoder = LabelEncoder()
encoder.fit(age_group_)
labels=encoder.transform(age_group_)
print(labels)


#gender

gender_=['Male', 'Female']

encoder = LabelEncoder()
encoder.fit(gender_)
labels = encoder.transform(gender_)

labels = labels.reshape(-1,1)

oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)


#marride

marride_=['1', '2', '3', '0']

encoder = LabelEncoder()
encoder.fit(marride_)
labels = encoder.transform(marride_)

labels = labels.reshape(-1,1)

oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)


#race

race_=['Asian', 'Arab', 'Black', 'Indigenous Australian', 'Native American', 'White', 'Other']

encoder = LabelEncoder()
encoder.fit(race_)
labels = encoder.transform(race_)

labels = labels.reshape(-1,1)

oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)

#religion

religion_=['Agnostic', 'Atheist', 'Buddhist', 'Christian_Catholic', 'Christian_Mormon', 'Christian_Protestant', 'Christian_Other', 'Hindu', 'Jewish', 'Muslim', 'Sikh', 'Other']

encoder = LabelEncoder()
encoder.fit(religion_)
labels = encoder.transform(religion_)

labels = labels.reshape(-1,1)

oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)

train_dummy.head(30)

In [ ]:
#원핫 인코딩 실험 2

race_n = ['Asian', 'Arab', 'Black', 'Indigenous Australian', 'Native American', 'White', 'Other']

data_p['race_Asian'] = data_p['race'] == 'Asian'

data_p[['race', 'race_Asian']].head(20)

def dummy_data(data, columns) :
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data

dummy_columns = ['race']
train_dummy = dummy_data(data_p, dummy_columns)

In [ ]:
#모델 평가 함수
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))
    

In [ ]:
# label/feature 분리 및 학습세트/평가세트 분리
y_labels = vote_df.loc[:,'voted']
X_features = vote_df.drop('voted', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=156)

# 분포 비율 확인
train_cnt = y_train.count()
test_cnt = y_test.count()

print("학습 세트 Shape:{0}, 테스트 세트 Shape:{1}".format(X_train.shape, X_test.shape))
print(" 학습 세트 레이블 값 분포 비율")
print(y_train.value_counts()/train_cnt)
print("\n 테스트 세트 레이블 값 분포 비율")
print(y_test.value_counts()/test_cnt)
print()

# 데이터 정규화(스케일링)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

test_sc = scaler.transform(test_df) # test data scaling



In [ ]:
#undersampling 진행
under_sampler = RandomUnderSampler(random_state=156)
X_train_under, y_train_under = under_sampler.fit_resample(X_train, y_train)



# 학습
evals = [(X_test, y_test)]
xgb_model = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3, objective="binary:logistic", random_state=156)
xgb_model.fit(X_train_under, y_train_under, early_stopping_rounds=100, eval_metric="logloss", eval_set=evals, verbose=True)
preds = xgb_model.predict(X_test)
pred_proba = xgb_model.predict_proba(X_test)[:,1]

get_clf_eval(y_test, preds, pred_proba)

In [ ]:
#KFOLD 진행
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

result_skfold = StratifiedKFold(n_splits=5)
idx_iter=0
cv_accuracy =[]

for train_index, test_index in result_skfold.split(X_features, y_labels):
    X_train_under, X_test = X_features[train_index], X_features[test_index]
    y_train_under, y_test = y_labels[train_index], y_labels[test_index]
    
    result_clf.fit(X_train_under, y_train_under)
    pred = result_clf(X_test)
    
    idx_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred),4 )
    train_size = X_train_under.shape[0]
    test_size = X_test.shape[0]
    
    cv_accuracy.append(accuracy)

In [ ]:
# 하이퍼파라미터 수정 학습과정 및 결과
evals = [(X_test, y_test)]
xgb_model = XGBClassifier(n_estimators=400,
                          learning_rate=0.1,
                          colsample_bytree= 1.0,
                          max_depth= 12,
                          min_child_weight= 5,
                          min_split_gain= 0.1,
                          num_leaves= 40.91732192323167,
                          reg_alpha= 0.5,
                          reg_lambda= 0.5,
                          subsample= 1)
xgb_model.fit(X_train_over, y_train_over, early_stopping_rounds=100, eval_metric="logloss", eval_set=evals, verbose=True)
w_preds = xgb_model.predict(X_test)
w_pred_proba = xgb_model.predict_proba(X_test)[:,1]

get_clf_eval(y_test, w_preds, w_pred_proba)


#도출되었던 최적 하이퍼파라미터 값
{'target': 0.9724714510197174,
 'params': {'colsample_bytree': 1.0,
  'max_depth': 12.0,
  'min_child_weight': 5.0,
  'min_split_gain': 0.1,
  'num_leaves': 40.91732192323167,
  'reg_alpha': 0.5,
  'reg_lambda': 0.5,
  'subsample': 1.0}}